In [8]:
from dat import Dat
data = Dat()
data.add_dict('wiki')
data.add_dict('twitter')
data.make_gen_dict()
X,y = data.getXy()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def jd(v,w): #Jaccard metric distance between two frames
    ci  = np.sum(v)
    cj  = np.sum(w)
    cij = np.sum((v + w)//2)
    if cij == 0:
        return(0)
    else:
        return(1 - (cij)/(ci + cj - cij))
    
def cs(v,w):
    return(cosine_similarity([v], [w])[0][0])
    
from sklearn.cluster import SpectralClustering

n_frames = y.shape[1]
frame_dist_matrix = np.zeros((n_frames, n_frames))
for i in range(0, n_frames):
    for j in range(i, n_frames):
        d = jd(y[:,i],y[:,j])
        frame_dist_matrix[i][j] = d
        frame_dist_matrix[j][i] = d
        
n_verbs = y.shape[0]
verb_syntax_dist_matrix = np.zeros((n_verbs, n_verbs))
for i in range(0, n_verbs):
    for j in range(i, n_verbs):
        d = jd(y[i],y[j])
        verb_syntax_dist_matrix[i][j] = d
        verb_syntax_dist_matrix[j][i] = d
        
verb_cossim_dist_matrix = np.zeros((n_verbs, n_verbs))
for i in range(0, n_verbs):
    for j in range(i, n_verbs):
        d = cs(X[i],X[j])
        verb_cossim_dist_matrix[i][j] = d
        verb_cossim_dist_matrix[j][i] = d

In [64]:
import scipy.io

scipy.io.savemat('fsd.mat', mdict={'frame_syntax_distance': frame_dist_matrix})
scipy.io.savemat('vsd.mat', mdict={'verb_syntax_distance': verb_syntax_dist_matrix})
scipy.io.savemat('vcd.mat', mdict={'verb_cossim_distance': verb_cossim_dist_matrix})

In [87]:
S1 = np.array(scipy.io.loadmat("si.mat")["S1"]).T[0]
S2 = np.array(scipy.io.loadmat("si.mat")["S2"]).T[0]
S3 = np.array(scipy.io.loadmat("si.mat")["S3"]).T[0]
S4 = np.array(scipy.io.loadmat("si.mat")["S4"]).T[0]
S5 = np.array(scipy.io.loadmat("si.mat")["S5"]).T[0]
S6 = np.array(scipy.io.loadmat("si.mat")["S6"]).T[0]
S7 = np.array(scipy.io.loadmat("si.mat")["S7"]).T[0]

consensus = np.zeros((292, 292))
f_list = []
for i in range(0, 291):
    for j in range(i, 291):
        b1 = int(S1[i] == S1[j])
        b2 = int(S2[i] == S2[j])
        b3 = int(S3[i] == S3[j])
        b4 = int(S4[i] == S4[j])
        b5 = int(S5[i] == S5[j])
        b6 = int(S6[i] == S6[j])
        b7 = int(S7[i] == S7[j])
        f = (b1 + b2 + b3 + b4 + b5 + b6 + b7)/7
        f_list += [f]
        consensus[i][j] = f
        consensus[j][i] = f

from sklearn.metrics import adjusted_mutual_info_score as ami
l = [S1, S2, S3, S4, S5, S6, S7]
ami_list = []
for i in range(0, 7):
    for j in range(i+1, 7):
        ami_list += [ami(l[i], l[j])]
        
np.mean(ami_list)

0.77103705785774412

In [ ]:
# Determing list of diagnostic frames, 
        

        
aff = np.exp(- frame_dist_matrix ** 2 / (2. * 1 ** 2)) #Gaussian heat kernel

n_diagnostic_frames = 50

sc_frames = SpectralClustering(n_clusters = n_diagnostic_frames, affinity = "precomputed")
frame_cluster_predictions = sc_frames.fit_predict(aff)
frame_cluster_pairs = list(enumerate(frame_cluster_predictions))

In [ ]:
from sklearn.cluster import KMeans
glove_verbs = list(map(lambda v : (v[0]), list(gen_dict.items())))
glove_vecs = np.array(list(map(lambda v : (v[1][1]), list(gen_dict.items()))))

n_diagnostic_verbs = 100

kmc_verbs = KMeans(n_clusters = n_diagnostic_verbs)
verb_cluster_predictions = kmc_verbs.fit_predict(glove_vecs)

verb_cluster_pairs = list(enumerate(verb_cluster_predictions))

In [ ]:
def cluster_sample(l, n_clusters):
    res = []
    random.shuffle(l)
    for i in range(0, n_clusters):
        for v,c in l:
            if i == c:
                bv = v
        res += [bv]
    return(res)
        
import pandas as pd

def maketable():
    diagnostic_verb_list = [glove_verbs[i] for i in cluster_sample(verb_cluster_pairs, n_diagnostic_verbs)]
    diagnostic_frame_list = [frame_list[i] for i in cluster_sample(frame_cluster_pairs, n_diagnostic_frames)]

    if len(diagnostic_verb_list) - len(list(set(diagnostic_verb_list))) > 0:
        return(maketable())
    elif len(diagnostic_frame_list) - len(list(set(diagnostic_frame_list))) > 0:
        return(maketable())
    else:
        data = np.zeros((n_diagnostic_verbs, n_diagnostic_frames))
        for i in range(0, n_diagnostic_verbs):
            for j in range(0, n_diagnostic_frames):
                data[i][j] = verbnet_dict[diagnostic_verb_list[i]][ frame_list.index(diagnostic_frame_list[j]) ]

        df = pd.DataFrame(data, index = diagnostic_verb_list, columns = diagnostic_frame_list, dtype=int)


        for i in range(0, n_diagnostic_verbs//2):
            verb = pd.Series.argmin(df.sum(axis=1))
            df = df.drop(verb, axis=0)

        for i in range(0, n_diagnostic_frames//2):
            frame = pd.Series.argmin(df.sum(axis=0))
            df = df.drop(frame, axis=1)

        return(df)

btable = maketable()
bcount = pd.DataFrame.sum(df.sum(axis=0))
for i in range(0, 100):
    t = maketable()
    count = pd.DataFrame.sum(t.sum(axis=0))
    if count > bcount:
        btable = t
        bcount = count
df = btable

print("Sparsity = ", pd.DataFrame.sum(df.sum(axis=0)) / (n_diagnostic_frames//2 * n_diagnostic_verbs//2))
print("Size =", df.shape)